**Objective**: Exploring methods for building a model for identifying eating activity in Capture24

In [36]:
import os
import numpy as np
import pandas as pd
from glob import glob
import scipy.stats as stats
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from joblib import Parallel, delayed
import urllib
import shutil
from tqdm.auto import tqdm
import utils  # helper functions -- check out utils.py
import zipfile
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import tabulate
from imblearn.over_sampling import SMOTE
import aiden_feature as af
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, GroupKFold
import output_utils
from sklearn.preprocessing import LabelEncoder

# For reproducibility
np.random.seed(42)

### Load data

In [25]:
# Load processed files
outputpath = os.path.expanduser("~/eating_detect/data/")
X = np.load(outputpath + 'processed_data/X.npy', mmap_mode='r')
Y = np.load(outputpath + 'processed_data/Y.npy')
T = np.load(outputpath + 'processed_data/T.npy')
pid = np.load(outputpath + 'processed_data/pid.npy')


# extract features
# X_feats = pd.DataFrame(Parallel(n_jobs=8)(delayed(af.aidan_features)(x, 100) for x in tqdm(X)))
# save extracted features
# X_feats.to_pickle(outputpath + 'processed_data/X_feats.pkl')

### Assign eating label

In [ ]:
# get the indices of labels that is potentially involved eating
eat_indices = np.array([index for index, element in enumerate(Y) if 'eat' in element])

# load the dictionary that maps the text labels to simplified labels
label_dict_path = os.path.expanduser("~/capture24/annotation-label-dictionary.csv")
anno_label_dict = pd.read_csv(
    label_dict_path,
    index_col='annotation', 
    dtype='string'
)
Y_simple = np.array([anno_label_dict.loc[y, 'label:Willetts2018'] for y in Y])

# get the unique labels related to eating
eating_labels = np.unique(Y[eat_indices])

# write the eating labels to a file for manual inspection
with open(outputpath + 'eating_labels.txt', 'w') as f:
    for item in eating_labels:
        f.write("%s\n" % item)

# Now I go the the eating_labels.txt file and manually select the labels that are related to eating


In [26]:
# after inspection, I have manually created a dictionary that maps the eating labels to simplified labels
eating_label_dict_path = os.path.expanduser("~/eating_detect/data/eating_labels_simple.tsv")
eating_label_dict = pd.read_csv(
    eating_label_dict_path,
    sep='\t',
    dtype='string'
)

# modify the Y_simple array to add eating-specific labels
# only replace with eating and maybe-eating lables, and ignore not-eating labels
Y_simple_eating = np.copy(Y_simple).astype('U12')
for i in eat_indices:
    label = Y[i]
    eating_label = eating_label_dict.loc[label, 'simple']
    if eating_label == 'eating':
        Y_simple_eating[i] = eating_label
        
# check the frequency of eating lables
print(pd.Series(Y_simple_eating).value_counts())

# remove records with the maybe-eating labels
#rm_ind = np.where(Y_simple_eating == 'maybe-eating')[0]
#X_simple_eating = np.delete(X, rm_ind, axis=0)
#Y_simple_eating = np.delete(Y_simple_eating, rm_ind, axis=0)
#T_simple_eating = np.delete(T, rm_ind, axis=0)
#pid_simple_eating = np.delete(pid, rm_ind, axis=0)

sleep        118802
sit-stand    111261
mixed         39398
walking       19971
vehicle       11580
eating         8728
bicycling      2990
Name: count, dtype: int64


### Set up data for ML models

In [27]:
# read the features extracted from the accelerometer data
X_feats = pd.read_pickle(outputpath + 'processed_data/X_feats.pkl')
#X_feats_eating = X_feats.drop(rm_ind, axis=0)

# Hold out participants P101-P151 for testing (51 participants)
test_ids = [f'P{i}' for i in range(101,152)]
mask_test = np.isin(pid, test_ids)
mask_train = ~mask_test
X_train, Y_train, T_train, pid_train = \
    X_feats[mask_train], Y_simple_eating[mask_train], T[mask_train], pid[mask_train]

X_test, Y_test, T_test, pid_test = \
    X_feats[mask_test], Y_simple_eating[mask_test], T[mask_test], pid[mask_test]
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (237106, 131)
Shape of X_test: (75624, 131)


### Random Forest

In [ ]:

# Argument oob_score=True to be used for HMM smoothing (see later below)
clf = BalancedRandomForestClassifier(
    n_estimators=1000,
    replacement=False,
    sampling_strategy='not minority',
    n_jobs=8,
    random_state=42,
    verbose=1,
    oob_score = False
)

**Grid Search**

In [ ]:
## searching for the best parameters for max_depth and max_features

param_grid = {
    'max_depth': [4, 6, 8, 10],
    'max_features': ['sqrt', 0.1, 0.2, 0.3, 0.4, 0.5 ]
}
group_kfold = GroupKFold(n_splits=2)
grid_clf_BA = GridSearchCV(clf, param_grid, cv=group_kfold, scoring='balanced_accuracy')
grid_clf_BA.fit(X_train, Y_train, groups=pid_train)
Y_test_pred = grid_clf_BA.predict(X_test)

# record performance
grid_clf_BA_metrics = output_utils.record_performance(grid_clf_eatBA.best_estimator_, Y_test, Y_test_pred, grid_clf.classes_[0])
grid_clf_BA_metrics['metrics']

# use the best parameters to train the model
clf = grid_clf_BA.best_estimator_
clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)

# record the performance
clf_BA_metrics = output_utils.record_performance(clf, Y_test, Y_test_pred, clf.classes_)
clf_BA_metrics['metrics']




**Grid Search - eating BA**

In [ ]:
## searching for the best parameters for max_depth and max_features

param_grid = {
    'max_depth': [4, 6, 8, 10],
    'max_features': ['sqrt', 0.1, 0.2, 0.3, 0.4, 0.5 ]
}

# custom scorer for balance accuracy of only "eating"
custom_scorer = utils.make_ba_scorer_for_class("eating")

# perform the grid search
group_kfold = GroupKFold(n_splits=2)
grid_clf_eatBA = GridSearchCV(clf, param_grid, scoring=custom_scorer, cv=group_kfold)
grid_clf_eatBA.fit(X_train, Y_train, groups=pid_train)
Y_test_pred = grid_clf_eatBA.predict(X_test)

# record the performance
grid_clf_eatBA_metrics = output_utils.record_performance(grid_clf_eatBA.best_estimator_, Y_test, Y_test_pred, grid_clf.classes_[0])
grid_clf_eatBA_metrics['metrics']

# use the best parameters to train the model
clf = grid_clf_eatBA.best_estimator_
clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)

# record the performance
clf_eatBA_metrics = output_utils.record_performance(clf, Y_test, Y_test_pred, clf.classes_)
clf_eatBA_metrics['metrics']


### Test Dynamic Time Warping

**Get the training data**

In [1]:
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

# for DTW the feature is the time series itself
X_train_dtw = X[mask_train]
X_test_dtw = X[mask_test]

NameError: name 'X' is not defined

**Down sample majority classes**

In [ ]:
# reload utils
import importlib
importlib.reload(utils)

In [ ]:
# set the target class
target_class = 'eating'

# get the frequency of each classes in the training set
class_freq = pd.Series(Y_train).value_counts()

# downsample the classes that have more samples
dnsmpl_classes = class_freq[class_freq > class_freq[target_class]].index
upsmpl_classes = class_freq[class_freq < class_freq[target_class]].index

# downsample the classes to be the same as the target class
Y_train_dtw_re_list = []
X_train_dtw_re_list = []

for cls in dnsmpl_classes:
    cls_indices = np.where(Y_train == cls)[0]
    y_train_cls = Y_train[cls_indices]
    downsampled_indices = utils.resampled_indices(y_train_cls, class_freq[target_class])
    y_train_cls_dnsmpl = y_train_cls[downsampled_indices]
    X_train_cls_dnsmpl = X_train_dtw[cls_indices[downsampled_indices]]
    Y_train_dtw_re_list.append(y_train_cls_dnsmpl)
    X_train_dtw_re_list.append(X_train_cls_dnsmpl)

for cls in upsmpl_classes:
    cls_indices = np.where(Y_train == cls)[0]
    y_train_cls = Y_train[cls_indices]
    upsampled_indices = utils.resampled_indices(y_train_cls, class_freq[target_class], replace=True)
    y_train_cls_upsmpl = y_train_cls[upsampled_indices]
    X_train_cls_upsmpl = X_train_dtw[cls_indices[upsampled_indices]]
    Y_train_dtw_re_list.append(y_train_cls_upsmpl)
    X_train_dtw_re_list.append(X_train_cls_upsmpl)

# add the target class
cls_indices = np.where(Y_train == target_class)[0]
y_train_cls = Y_train[cls_indices]
X_train_cls = X_train_dtw[cls_indices]
Y_train_dtw_re_list.append(y_train_cls)
X_train_dtw_re_list.append(X_train_cls)

# Convert the lists to NumPy arrays
Y_train_dtw_re = np.concatenate(Y_train_dtw_re_list)
X_train_dtw_re = np.vstack(X_train_dtw_re_list)


# print frequency of each class
print(pd.Series(Y_train_dtw_re).value_counts())

In [40]:
regressor = KNeighborsTimeSeriesRegressor(algorithm="kd_tree")

In [41]:
# encode the labels to integers

label_encoder = LabelEncoder()
label_encoder.fit(Y_train_dtw_re)
Y_train_dtw_re_coded = label_encoder.transform(Y_train_dtw_re)
Y_test_coded = label_encoder.transform(Y_test)

regressor.fit(X_train_dtw_re, Y_train_dtw_re_coded)

KeyboardInterrupt: 

In [ ]:
regressor.fit(X_train_dtw, Y_train)

### Perform HMM smoothing
**Use grid search to find the best HMM emission matrix**

In [ ]:
# Use the conveniently provided out-of-bag probability predictions from the
# random forest training process.
Y_train_prob = clf.oob_decision_function_  # out-of-bag probability predictions
labels = clf.classes_  # need this to know the label order of cols of Y_train_prob
hmm_params = utils.train_hmm(Y_train_prob, Y_train, labels)  # obtain HMM matrices/params
Y_test_pred_hmm = utils.viterbi(Y_test_pred, hmm_params)  # smoothing
print('\nClassifier performance -- HMM smoothing')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_hmm))

# Check again participant
mask = pid_test == 'P101'
fig, ax = utils.plot_compare(T_test[mask],
                             Y_test[mask],
                             Y_test_pred_hmm[mask])
fig.show()


In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
from sklearn.preprocessing import LabelEncoder
#X_resampled, y_resampled = smote.fit_resample(X_train, Y_train)
label_encoder = LabelEncoder()
y_resampled_encoded = label_encoder.fit_transform(y_resampled)
xgb_clf = xgb.XGBClassifier(
    n_estimators=1000,
    use_label_encoder=True,
    eval_metric='logloss',  # Added to avoid a warning about the default metric
    n_jobs=8,
    verbosity=1
)
xgb_clf.fit(X_resampled, y_resampled_encoded)


In [ ]:
y_pred_encoded = xgb_clf.predict(X_test)
Y_test_pred = label_encoder.inverse_transform(y_pred_encoded)

print("Accuracy:", accuracy_score(Y_test, y_pred))
print("\nClassification Report:\n", classification_report(Y_test, Y_test_pred))

# Check again participant
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                              Y_test[mask],
                              Y_test_pred[mask])
fig.show()


In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, Y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=100,
    oob_score=True,
    n_jobs=8,
    random_state=42,
    verbose=1
)

rf_clf.fit(X_resampled, y_resampled)

In [ ]:
Y_test_pred = rf_clf.predict(X_test)
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred))

# Check again participant
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                              Y_test[mask],
                              Y_test_pred[mask])
fig.show()

In [ ]:
X_resampled.shape

### Train a random forest classifier

In [ ]:
# Argument oob_score=True to be used for HMM smoothing (see later below)
clf = BalancedRandomForestClassifier(
    n_estimators=1000,
    replacement=False,
    sampling_strategy='not minority',
    n_jobs=8,
    random_state=42,
    verbose=1,
)
clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)
balanced_accuracy_score(Y_test, Y_test_pred)

print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred))


In [ ]:
# frequency of each label
pd.Series(Y_test_pred).value_counts()

In [ ]:
## searching for the best parameters for max_depth and max_features
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, GroupKFold

custom_scorer = make_ba_scorer_for_class("eating")

param_grid = {
    'max_depth': [4, 6, 8, 10],
    'max_features': ['sqrt', 0.1, 0.2, 0.3, 0.4, 0.5 ]
}


group_kfold = GroupKFold(n_splits=2)
grid_clf = GridSearchCV(clf, param_grid, scoring=custom_scorer, cv=group_kfold)
grid_clf.fit(X_train, Y_train, groups=pid_train)
# check best parameters
print(grid_clf.best_params_)
# max_depth=6

# check the performance of all the parameters
Y_test_pred = grid_clf.predict(X_test)
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred))

# get the balanced accuracy for class 'eating'
mask = Y_test == 'eating'
# classify Y_test into eating non-eating
Y_test_eating = np.copy(Y_test)
Y_test_eating[mask] = 'eating'
Y_test_eating[~mask] = 'non-eating'
# classify Y_test_pred into eating non-eating
mask = Y_test_pred == 'eating'
Y_test_pred_eating = np.copy(Y_test_pred)
Y_test_pred_eating[mask] = 'eating'
Y_test_pred_eating[~mask] = 'non-eating'
balanced_accuracy_score(Y_test_eating, Y_test_pred_eating)

# caluculate the specificty for class 'non-eating'
recall_score(Y_test_eating, Y_test_pred_eating, labels=['eating'], average=None)

# get the best estimator
clf = grid_clf.best_estimator_.set_params(bootstrap=False)
clf = grid_clf.best_estimator_.set_params(replacement=False)
clf = grid_clf.best_estimator_.set_params(oob_score=True)
# fit the best estimator
clf.fit(X_train, Y_train)
# predict
Y_test_pred = clf.predict(X_test)
# HMM smoothing
# Use the conveniently provided out-of-bag probability predictions from the
# random forest training process.
Y_train_prob = clf.oob_decision_function_  # out-of-bag probability predictions
labels = clf.classes_  # need this to know the label order of cols of Y_train_prob
hmm_params = utils.train_hmm(Y_train_prob, Y_train, labels, True)  # obtain HMM matrices/params
Y_test_pred_hmm = utils.viterbi(Y_test_pred, hmm_params)  # smoothing
print('\nClassifier performance -- HMM smoothing')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_hmm))

# detatch and reload utils.py
import importlib
importlib.reload(utils)



# Check again participant
mask = pid_test == 'P101'
fig, ax = utils.plot_compare(T_test[mask],
                             Y_test[mask],
                             Y_test_pred_hmm[mask])
fig.show()






In [ ]:
# save the model for later use
import pickle




In [ ]:
# get the balanced accuracy for class 'eating'
mask = Y_test == 'eating'
# classify Y_test into eating non-eating
Y_test_eating = np.copy(Y_test)
Y_test_eating[mask] = 'eating'
Y_test_eating[~mask] = 'non-eating'
# classify Y_test_pred into eating non-eating
mask = Y_test_pred_hmm == 'eating'
Y_test_pred_eating = np.copy(Y_test_pred_hmm)
Y_test_pred_eating[mask] = 'eating'
Y_test_pred_eating[~mask] = 'non-eating'
balanced_accuracy_score(Y_test_eating, Y_test_pred_eating)


In [ ]:
def mode(alist):
    ''' Mode of a list, but return middle element if ambiguous '''
    m, c = stats.mode(alist)
    m, c = m.item(), c.item()
    if c==1:
        return alist[len(alist)//2]
    return m

def rolling_mode(t, y, window_size='100S'):
    y_dtype_orig = y.dtype
    # Hack to make it work with pandas.Series.rolling()
    y = pd.Series(y, index=t, dtype='category')
    y_code_smooth = y.cat.codes.rolling(window_size).apply(mode, raw=True).astype('int')
    y_smooth = pd.Categorical.from_codes(y_code_smooth, dtype=y.dtype)
    y_smooth = np.asarray(y_smooth, dtype=y_dtype_orig)
    return y_smooth

# Smooth the predictions of each participant
Y_test_pred_smooth = []
unqP, indP = np.unique(pid_test, return_index=True)
unqP = unqP[np.argsort(indP)]  # keep the order or else we'll scramble our arrays
for p in unqP:
    mask = pid_test == p
    Y_test_pred_smooth.append(rolling_mode(T_test[mask], Y_test_pred[mask]))
Y_test_pred_smooth = np.concatenate(Y_test_pred_smooth)

print('\nClassifier performance -- mode smoothing')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_smooth))

# Check again participant
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                              Y_test[mask],
                              Y_test_pred_smooth[mask])
fig.show()


In [ ]:
cv_results = grid_clf.cv_results_

# cv_results is a dictionary where each key is a string and each value is an array.
# The keys are metrics and the values are the results for each hyperparameter combination.

# For example, to print the mean test score for each parameter combination:
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(params, '->', mean_score)

### Model performance


In [ ]:
import importlib
importlib.reload(utils)

In [ ]:
Y_test_pred = clf.predict(X_test)
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred))

# Check again participant
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                              Y_test[mask],
                              Y_test_pred[mask])
fig.show()


In [ ]:
X_feats.shape

### Rolling mode smoothing

In [ ]:
def mode(alist):
    ''' Mode of a list, but return middle element if ambiguous '''
    m, c = stats.mode(alist)
    m, c = m.item(), c.item()
    if c==1:
        return alist[len(alist)//2]
    return m

def rolling_mode(t, y, window_size='100S'):
    y_dtype_orig = y.dtype
    # Hack to make it work with pandas.Series.rolling()
    y = pd.Series(y, index=t, dtype='category')
    y_code_smooth = y.cat.codes.rolling(window_size).apply(mode, raw=True).astype('int')
    y_smooth = pd.Categorical.from_codes(y_code_smooth, dtype=y.dtype)
    y_smooth = np.asarray(y_smooth, dtype=y_dtype_orig)
    return y_smooth

# Smooth the predictions of each participant
Y_test_pred_smooth = []
unqP, indP = np.unique(pid_test, return_index=True)
unqP = unqP[np.argsort(indP)]  # keep the order or else we'll scramble our arrays
for p in unqP:
    mask = pid_test == p
    Y_test_pred_smooth.append(rolling_mode(T_test[mask], Y_test_pred[mask]))
Y_test_pred_smooth = np.concatenate(Y_test_pred_smooth)

print('\nClassifier performance -- mode smoothing')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_smooth))

# Check again participant
mask = pid_test == 'P101'
fig, axs = utils.plot_compare(T_test[mask],
                              Y_test[mask],
                              Y_test_pred_smooth[mask])
fig.show()


### Hidden Markov Model

In [ ]:
import importlib
importlib.reload(utils)
# Use the conveniently provided out-of-bag probability predictions from the
# random forest training process.
Y_train_prob = clf.oob_decision_function_  # out-of-bag probability predictions
labels = clf.classes_  # need this to know the label order of cols of Y_train_prob
hmm_params = utils.train_hmm(Y_train_prob, Y_train, labels)  # obtain HMM matrices/params
Y_test_pred_hmm = utils.viterbi(Y_test_pred, hmm_params)  # smoothing
print('\nClassifier performance -- HMM smoothing')
print('Out of sample:\n', metrics.classification_report(Y_test, Y_test_pred_hmm))

# Check again participant
mask = pid_test == 'P101'
fig, ax = utils.plot_compare(T_test[mask],
                             Y_test[mask],
                             Y_test_pred_hmm[mask])
fig.show()


In [ ]:
probabilities = xgb_clf.predict_proba(X_test)
threshold = 0.4  # Example threshold
y_pred_adjust = (probabilities[:, 1] >= threshold).astype(int)
y_pred_adjust = label_encoder.inverse_transform(y_pred_adjust)
print('Out of sample:\n', metrics.classification_report(Y_test, y_pred_adjust))


In [ ]:
y_pred_adjust
